In [ ]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
from sklearn.impute import SimpleImputer

In [ ]:
#importing the incident dataset for analysis
incident=pd.read_csv("./Data/incident_event_log.csv")
incident.head()

In [ ]:
#Analysing the shape of the dataframe
incident.shape

In [ ]:
#Calculating NA's in each column
incident.isnull().sum(axis=0) 

In [ ]:
#Checking for the datatypes present in the dataframe
incident.dtypes

In [ ]:
#Estimating the range of values in the dataset
incident.describe()

In [ ]:
#df.drop(['cmdb_ci', 'problem_id', 'rfc', 'vendor','caused_by','caller_id','opened_by','sys_created_by','sys_updated_by','location','resolved_by','notify','assigned_to','assignment_group'],axis = 1,inplace=True)

In [ ]:
 #Replacing ? in the dataframe
incident.replace('?', np.NaN, inplace = True)
incident.info()

In [ ]:
#Generating a heatmap to explore the columns which have most NA values
fig, ax = plt.subplots(figsize=(10,10))  
sns.heatmap(incident.isnull(), cmap='viridis',ax=ax) 

In [ ]:
#Removing the columns which had NA values the most
incident_processed = incident.drop(incident.columns[[19,28,29,30,31]],axis=1) 

In [ ]:
#Replacing with mean for numerical columns
df_imputer=SimpleImputer(missing_values=np.NaN, strategy='mean', verbose=0)
df_imputer=df_imputer.fit(incident_processed.iloc[:,[3,4,5]])# columns with missing values
incident_processed.iloc[:,[3,4,5]]=df_imputer.transform(incident_processed.iloc[:,[3,4,5]])

In [ ]:
#Replacing with mode for categorical columns
for column in ['caller_id', 'opened_by', 'sys_created_by','sys_created_at','location','category','subcategory','u_symptom','assignment_group','assigned_to','closed_code','resolved_by','resolved_at']:
    incident_processed[column].fillna(incident_processed[column].mode()[0], inplace=True) 

In [ ]:
# Applying labelencoder to transform categorical value to numerical value
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = incident_processed.apply(le.fit_transform) 

In [ ]:
df.head(8)

In [ ]:
#remove outliers
#df = df[df['incident_state'] != '-100' ]

In [ ]:
# replace subtypes of Awaiting incident state into main type of Awaiting
df['incident_state'].replace(['Awaiting User Info','Awaiting Problem','Awaiting Vendor','Awaiting Evidence'],'Awaiting',inplace = True)

In [ ]:
df[['priority','incident_state']].groupby('incident_state').agg(['count'])

In [ ]:
# df['active'] = df['active'].apply(lambda x : 1 if x == 'True' else 0)
# df['made_sla'] = df['made_sla'].apply(lambda x : 1 if x == 'True' else 0)
# df['knowledge'] = df['knowledge'].apply(lambda x : 1 if x == 'True' else 0)
# df['u_priority_confirmation'] = df['u_priority_confirmation'].apply(lambda x : 1 if x == 'True' else 0)
# df['priority'] = df['priority'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# df['impact'] = df['impact'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# df['urgency'] = df['urgency'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# reass_count_labels = [1,2,3,4,5]
# df['reassignment_count'] = pd.cut(df['reassignment_count'],bins = 5, labels = reass_count_labels)
# df = pd.get_dummies(df,columns = ['incident_state'])

In [ ]:
#Convert date time string values into pandas datetime values
df['closed_at'] = pd.to_datetime(df['closed_at'])
df['opened_at'] = pd.to_datetime(df['opened_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'])
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])


In [ ]:
#calculate features like lifetime = time the incident was active, action_at = time taken when first action was undertaken
df['lifetime'] = df['closed_at'] - df['opened_at']
df['action_at'] = df['sys_updated_at'] - df['opened_at']

In [ ]:
#extract only the day values from datetime value for features
df['lifetime'] = df['lifetime'].dt.days
df['action_at'] = df['action_at'].dt.days

In [ ]:
#check number of records having close time before open time (which is wrong : It is because the the record opentime was updated by the user. Hence we extract the original open time by sorting cases with similar case number in ascending order and taking the opentime from the record with earliest system updated time)
df['number'][df['lifetime'] < 0], df['opened_at'][df['lifetime'] < 0]

In [ ]:
def update_same_case_records(df):
    df.reset_index()
    unique_case_ids = df['number'].unique()
    if len(unique_case_ids) > 0:
        for case_id in tqdm(unique_case_ids):
            #indicidual case id reccords
            df_case = df[df['number'] == case_id]
            #sort the column by system updated at
            sorted_df_case = df_case.sort_values('sys_updated_at')
            #sorted_df_case.reset_index(inplace=True)
            initial_record = sorted_df_case.iloc[[0]]
            latest_record = sorted_df_case.iloc[[sorted_df_case.shape[0]-1]]
            lifetime = latest_record['closed_at'] - latest_record['opened_at']
            lifetime = lifetime.dt.days.values[0]
            if  lifetime <= 0: latest_record['opened_at'] = initial_record['sys_updated_at'].values
            df = df[df['number'] != case_id]
            if df['number'].isnull().values.any():
                df = pd.concat([df,latest_record.to_frame()])
            else: df = pd.concat([df,latest_record])
    return df


In [ ]:
df_same_case = df[df['lifetime'] < 0]
df_unique_case = df[df['lifetime'] > 0]

In [ ]:
df_same_case = update_same_case_records(df_same_case)

In [ ]:
#merge unique cases and the newly created df having single record for every case number
df_new = pd.concat([df_unique_case,df_same_case])

In [ ]:
# create features lifetime and action_at
df_new['lifetime'] = df_new['closed_at'] - df_new['opened_at']
df_new['action_at'] = df_new['sys_updated_at'] - df_new['opened_at']

#store only days values as features
df_new['lifetime'] = df_new['lifetime'].dt.days
df_new['action_at'] = df_new['action_at'].dt.days

#scale lifetime and action_at features
features = ['lifetime','action_at']
df_new[features] = scaler.fit_transform(df_new[features])

In [ ]:
df_new.head()

In [ ]:
#check if there are records with negative lifetime
df_new['number'][df_new['lifetime'] < 0], df_new['opened_at'][df_new['lifetime'] < 0]

In [ ]:
# store case numbers for analytical purpose
ids = df_new['number']
df_new.drop(['number'],inplace = True, axis = 1)

In [ ]:
df_new.columns

In [ ]:
# drop irrelevant features
df_new.drop(['closed_at','opened_at','sys_updated_at','sys_created_at','resolved_at','closed_code','category','subcategory','u_symptom','contact_type'],axis = 1,inplace=True)

In [ ]:
# reset index values and drop index column for the model input
df_new.reset_index(inplace= True)
df_new.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_new.head()

In [ ]:
df_new.to_csv('Data/reopened.csv')